In [3]:
print("Hello there jupyter!")

Hello there jupyter!


In [2]:
!pip install opencv-python

In [16]:
!pip install pillow 

In [4]:
import pandas as pd
import numpy as np
import cv2
import PIL
import os
import pickle


In [5]:
def startcamera():
    video = cv2.VideoCapture(0)
    while True:
        ret, frame = video.read()
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        cv2.imshow("Attendence Frame", gray_frame)
        k = cv2.waitKey(1)
        if k == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

In [19]:

def startstoreface():
    face_data = []
    i=0
    names = []
    input_name = input("Please Enter Your Name: ")
    
    
    video = cv2.VideoCapture(0)
    detected_face = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    while True:
        ret, frame = video.read()
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detected_face.detectMultiScale(gray_frame,1.3,5)
        for (x,y,w,h) in faces:
            crp_img = frame[y:y+h,x:x+h,:]
            resized_img = cv2.resize(crp_img,(50,50))
            if len(face_data)<=100 and i%2 == 0:
                face_data.append(resized_img)
            i =i+ 1
            cv2.putText(frame,str(len(face_data)),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(50,50,255),1)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,50,55),2)
        cv2.imshow("Attendence Frame", frame)
        k = cv2.waitKey(1)
        if k == ord('q') or len(face_data) == 100:
            break
    face_data = np.asarray(face_data)
    face_data = face_data.reshape(100,-1)
    #Storing Name Labels
    if 'names.pkl' not in os.listdir('data/'):
        with open('data/names.pkl','wb') as f:
            names = [input_name]*100
            pickle.dump(names,f)
    else:
        with open('data/names.pkl','rb') as f:
            name = pickle.load(f)
        name = name + [input_name]*100
        with open('data/names.pkl','wb') as f:
            pickle.dump(name,f)
    #Storing Faces
    if 'face_data.pkl' not in os.listdir('data/'):
        with open('data/face_data.pkl','wb') as f:
            pickle.dump(face_data,f)
    else:
        with open('data/face_data.pkl','rb') as f:
            faces = pickle.load(f)
        faces = np.append(faces,face_data,axis=0)
        with open('data/face_data.pkl','wb') as f:
            pickle.dump(faces,f)

    video.release()
    cv2.destroyAllWindows()

In [30]:
startstoreface()

Please Enter Your Name: Sanjay


In [21]:
with open('data/names.pkl','rb') as f:
    name = pickle.load(f)
name = np.asarray(name)
z = list(set(name))
z

['Yash']

In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [11]:
from keras.applications import VGG16

In [15]:
modelT =  VGG16(weights='imagenet',include_top=False,input_shape=(50,50,3))
for layer in model.layers:
    layer.trainable=False

In [16]:
# model = Sequential([
#     modelT,
#     layers.Flatten(),
#     layers.Dense(128,activation='relu'),
#     layers.Dense(len(z),activation='softmax'),



# ])

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])


# with open('data/names.pkl','rb') as f:
#     Name = pickle.load(f)
# with open('data/face_data.pkl','rb') as f:
#     Face = pickle.load(f)
    
# Face = np.asarray(Face)
# Face = Face/255
# Name = np.asarray(Name)
# model.fit(Face,Name,epochs=30)
from sklearn.neighbors import KNeighborsClassifier

In [31]:
with open('data/names.pkl','rb') as f:
    Name = pickle.load(f)
with open('data/face_data.pkl','rb') as f:
    Face = pickle.load(f)
Knn = KNeighborsClassifier(n_neighbors=3)
Knn.fit(Face,Name)

KNeighborsClassifier(n_neighbors=3)

In [28]:
def predictface(model):
    video = cv2.VideoCapture(0)
    with open('data/names.pkl','rb') as f:
        Name = pickle.load(f)
    with open('data/face_data.pkl','rb') as f:
        Face = pickle.load(f)
    detected_face = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    while True:
        ret, frame = video.read()
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detected_face.detectMultiScale(gray_frame,1.3,5)
        for (x,y,w,h) in faces:
            crp_img = frame[y:y+h,x:x+h,:]
            resized_img = cv2.resize(crp_img,(50,50)).flatten().reshape(1,-1)
            output = model.predict(resized_img)
            cv2.putText(frame,str(output[0]),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,50,50),1)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,50,55),2)
        cv2.imshow("Attendence Frame", frame)
        k = cv2.waitKey(1)
        if k == ord('q'):
            break


    video.release()
    cv2.destroyAllWindows()

In [36]:
predictface(Knn)

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be ac

C:\Users\Yashkumar Dubey\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
